In [82]:
# Ataque de Lagarias al criptosistema de Merkle-Hellman

In [83]:
# Importamos el método de Merkle-Hellman
load("Merkle-Hellman_iterativo.py")

In [84]:
# Genera la matriz necesaria para aplicar Lagarias
# paso == 0 indica generar la matriz inicial, paso != 0 indica generar la matriz del caso 4
def generarMatriz(pk, s, paso):
    n = len(pk)
    filas = []
    
    # generamos los n primeros vectores
    for i in range(0, n):
        aux = zero_vector(n+1)
        aux[i] = 1
        aux[n] = pk[i]*(-1)
        filas.append(aux)

    # generamos el vector n+1
    b = zero_vector(n+1)
    if paso == 0:
        b[n] = s
    else:
        b[n] = sum(pk) - s
    filas.append(b)
    
    return Matrix(filas)

In [85]:
# Encuentra una solución en la matriz recibida
def buscarSolucion(matriz, pk, s):
    n = matriz.nrows()
    sol_encontrada = False
    solucion = []
    
    # compruebo si alguna fila es solución sin aplicar nada
    for j in range(0, n):
        if sol_encontrada == False:
            suma = 0
            fila = matriz[j]
        
            for i in range(0, len(fila)-1):
                if fila[i] < 0:
                    break
                suma += pk[i]*fila[i]
            if suma == s:
                sol_encontrada = True
                solucion = fila[:-1]
        
    # compruebo si alguna fila es solución tras dividir por un lambda fijo
    if sol_encontrada == False:
        for j in range(0, n):
            if sol_encontrada == False:
                suma = 0
                den_encontrado = False
                fila = matriz[j]
                den = 1
                
                for i in range(0, len(fila)-1):
                    if fila[i] != 0 and den_encontrado == False:
                        den = fila[i]
                        den_encontrado = True
                    aux = fila[i] / den
                    if aux < 0:
                        break
                    suma += pk[i]*aux
                    solucion.append(aux)
                if suma == s:
                    sol_encontrada = True
                else:
                    solucion = []

    solucion = list(solucion)
    return solucion

In [86]:
# Intercambia los 0 por 1 en una matriz
def cambiarMatriz(matriz):
    matriz_res = copy(matriz)

    for i in range(matriz.nrows()):
        for j in range(matriz.ncols()-1):
            if matriz[i, j] == 0:
                matriz_res[i, j] = 1
            elif matriz[i, j] == 1:
                matriz_res[i, j] = 0
    
    return matriz_res

In [87]:
# Aplica el ataque de Lagarias
def ataqueLagarias(pk, s):
    n = len(pk)
    encontrado = False
    solucion = []
    
    # generamos la matriz
    matriz_ini = generarMatriz(pk, s, 0)
    
    # aplicamos LLL
    matriz_res = matriz_ini.LLL()
    
    # buscamos una solución
    solucion = buscarSolucion(matriz_res, pk, s)
    
    # cambiamos los 0 por 1
    if len(solucion) == 0:
        matriz_cambio = cambiarMatriz(matriz_res)
        solucion = buscarSolucion(matriz_cambio, pk, s)
    
    # caso 4 del algoritmo
    if len(solucion) == 0:
        matriz_ini = generarMatriz(pk, s, 1)
        matriz_res = matriz_ini.LLL() 
        solucion = buscarSolucion(matriz_res, pk, s)
        
        # cambiamos los 0 por 1
        if len(solucion) == 0:
            matriz_cambio = cambiarMatriz(matriz_res)
            solucion = buscarSolucion(matriz_cambio, pk, s)
        
    return solucion

In [88]:
# Calcula el número de errores cometidos
def comprobarErrores(men_orig, men_obt):
        n = len(men_orig)
        vector_dif = []
        
        for i in range(0, n):
            vector_dif.append(abs(men_orig[i] - men_obt[i]))

        return sum(vector_dif)

In [89]:
# Main

# ---------- descomentar para ejecutarlo 1 vez ----------

# pk = merkle_hellman.pk
# s  = merkle_hellman.s

# lagarias = ataqueLagarias(pk, s)

# print()
# print("Clave pública      :", merkle_hellman.pk)
# print("Mensaje original   :", merkle_hellman.mensaje)
# print("Mensaje cifrado    :", merkle_hellman.s)
# print("Mensaje descifrado :", lagarias)
# print("Errores totales    :", comprobarErrores(merkle_hellman.mensaje, lagarias))

# ---------- descomentar para ejecutarlo p veces ----------

# variables ejecución
p   = 10  # número de criptosistemas a ejecutar
tam = 100  # tamaño del mensaje original
it  = 0   # número de veces aplicado el método iterativo

# variables internas
errores  = 0
err_long = 0
vacios   = 0
densidad = 0

# anaálisis de las p ejecuciones
print("Iteración \t Densidad \t\tResultado")
for i in range(0, p):
    print(i+1, end = "")
    merkle_hellman = Merkle_Hellman(tam, it)
    merkle_hellman.do()
    densidad = RR(merkle_hellman.tamano / log(max(merkle_hellman.pk), 2))
    print("\t\t", densidad, end="")
    lagarias = ataqueLagarias(merkle_hellman.pk, merkle_hellman.s)
    if len(lagarias) == 0:
        vacios += 1
        print("\tvacio")
    elif len(lagarias) != len(merkle_hellman.mensaje):
        err_long += 1
        print("\terror longitud")
    else:
        valor = comprobarErrores(merkle_hellman.mensaje, lagarias)
        errores += valor
        if valor != 0:
            print("\terror valor")
        else:
            print("\tobtenido")
            
print("\nErrores totales  tras", p, "iteraciones :", errores)
print("Vacios  totales  tras", p, "iteraciones :", vacios)
print("Errores longitud tras", p, "iteraciones :", err_long)

Iteración 	 Densidad 		Resultado
1		 0.496994712196399	vacio
2		 0.497494981280230	obtenido
3		 0.495411429031387	vacio
4		 0.497011471436489

KeyboardInterrupt: 